In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm
import sys
sys.path.insert(0,'../')
%load_ext autoreload
%autoreload 2
from utils import citibike_helpers
import numpy as np
import pandas as pd  #requirement comes with anaconda
import datetime 
from datetime import datetime as dt
import seaborn as sns
import operator
import networkx as nx
import community #requires separate install -  pip install python-louvain
from geopy.distance import vincenty # requires separate install - pip install geopy
from utils import nyctaxi_helpers
import warnings
warnings.filterwarnings('ignore')

In [2]:
inputfile="../datasets/citibike/201701-citibike-tripdata.csv.gz"
df = citibike_helpers.load_citibike_data(inputfile)
df = citibike_helpers.calculate_trip_durations_citibike(df)
#Here we focus on a subset of "interesting" trips with duration between 5 and 120 minutes.

In [3]:
df['Start Time'] =  pd.to_datetime(df['Start Time'], format='%Y-%m-%d %H:%M:%S')
df['Start Time Hour'] = pd.DatetimeIndex(df['Start Time']).hour
df['Start Time Minute'] = pd.DatetimeIndex(df['Start Time']).minute
df['Start Time Day of Week'] = pd.DatetimeIndex(df['Start Time']).dayofweek
f = lambda x: vincenty((x['Start Station Latitude'],x['Start Station Longitude']),(x['End Station Latitude'],x['End Station Longitude'])).miles
df['distance'] = df.apply(f,axis=1)


In [7]:
subset_trips=df[(df['Trip Duration Minutes']>5) & (df['Trip Duration Minutes']<120)]
subset_trips.drop(['Trip Duration', 'Start Time', 'Stop Time', 'Start Station ID','Start Station Name','End Station Name','End Station ID', 'Bike ID'],axis=1,inplace=True)
subset_trips = pd.get_dummies(subset_trips)
subset_trips.dropna(inplace=True)
subset_trips.head(2)

,Start Station Latitude,Start Station Longitude,End Station Latitude,End Station Longitude,Birth Year,Gender,Trip Duration Minutes,Start Time Hour,Start Time Minute,Start Time Day of Week,distance,User Type_Customer,User Type_Subscriber
0,40.782750,-73.971370,40.775794,-73.976206,1965.0,2,11.333333,0,0,6,0.542899,0,1
1,40.729236,-73.990868,40.748549,-73.988084,1987.0,2,21.383333,0,0,6,1.340572,0,1


In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

models = {}
lnrmodel = LinearRegression()
knnmodel = KNeighborsRegressor()
models.update({'Linear':lnrmodel})
models.update({'KNN':knnmodel})

In [9]:
y_cls = subset_trips['Trip Duration Minutes']
# move all the predictors into another variable
XX_cls = subset_trips.drop(['Trip Duration Minutes'], axis = 1)
# model name which could be any of the following
# sorted by the amount of time they take to train the model:
# Linear Regression, KNN Regression
for model_name, model in models.items():
    print("Running the {} Regression...".format(model_name))
    x_train, x_test, y_train, y_test = train_test_split(XX_cls, y_cls, test_size=0.33, random_state=42)
    y_pred = nyctaxi_helpers.run(model,x_train, y_train, x_test)
    r2 = r2_score(y_test, y_pred)
    print('R2 score={}'.format(r2))

Running the Linear Regression...
R2 score=0.5192549210155281
Running the KNN Regression...
R2 score=0.40007019712257696
